In [32]:
# --- 1. Install Required Library ---
# This command installs the official Google Cloud BigQuery library.
# The `--quiet` flag reduces the amount of installation output.
print("🔧 Installing google-cloud-bigquery library...")
!pip install --upgrade google-cloud-bigquery --quiet

# --- 2. Authenticate to Google Cloud ---
# This is the standard way to authenticate in Google Colab.
# It will open a pop-up window for you to log in and grant permissions.
from google.colab import auth
print("🔐 Authenticating user...")
auth.authenticate_user()
print("✅ Authentication successful.")

# --- 3. Load Data into BigQuery ---
from google.cloud import bigquery

# Construct a BigQuery client object.
# This will use the credentials from the authentication step.
client = bigquery.Client()

# Set variables for the BigQuery load job
project_id = 'qwiklabs-gcp-02-3205ce60aeba'
dataset_id = 'aero_alerts'
table_id = 'airports'
file_uri = 'gs://labs.roitraining.com/data-to-ai-workshop/airports.csv'

# --- Create the dataset if it doesn't exist ---
dataset_full_id = f"{project_id}.{dataset_id}"
print(f"Ensuring dataset '{dataset_full_id}' exists...")
dataset = bigquery.Dataset(dataset_full_id)
# You can specify a location, e.g., "US", "EU", "asia-northeast1"
# If you don't, it will be created in the multi-region location of the project.
dataset.location = "US"
client.create_dataset(dataset, exists_ok=True)
print(f"✅ Dataset is ready.")

# Construct the full table reference path
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Configure the load job
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    autodetect=True,      # Automatically infer the schema from the CSV
    skip_leading_rows=1,  # Skip the header row
)

# Start the load job
print(f"\n🚀 Starting job to load data from {file_uri} into {table_ref}...")
load_job = client.load_table_from_uri(
    file_uri,
    table_ref,
    job_config=job_config
)

# Wait for the job to complete
load_job.result()
print("✅ Job finished.")

# Get destination table info and print results
destination_table = client.get_table(table_ref)
print(f"🎉 Successfully loaded {destination_table.num_rows} rows into table '{table_id}'.")

🔧 Installing google-cloud-bigquery library...
🔐 Authenticating user...
✅ Authentication successful.
Ensuring dataset 'qwiklabs-gcp-02-3205ce60aeba.aero_alerts' exists...
✅ Dataset is ready.

🚀 Starting job to load data from gs://labs.roitraining.com/data-to-ai-workshop/airports.csv into qwiklabs-gcp-02-3205ce60aeba.aero_alerts.airports...
✅ Job finished.
🎉 Successfully loaded 746037 rows into table 'airports'.


In [33]:
# --- 1. Define the BigQuery SQL Query ---
# This query selects the identifier and coordinates for airports that are
# classified as 'large_airport' and are located in the 'US'.
sql_query = """
    SELECT
        iata_code,
        name,
        latitude_deg,
        longitude_deg
    FROM
        `qwiklabs-gcp-02-3205ce60aeba.aero_alerts.airports`
    WHERE
        type = 'large_airport' AND iso_country = 'US'
"""

# --- 2. Execute the Query ---
print("✈️ Fetching list of large US airports from BigQuery...")
try:
    # Execute the query and load the results directly into a pandas DataFrame
    airports_df = client.query(sql_query).to_dataframe()
    print(f"✅ Found {len(airports_df)} large airports.")
    # Display the first few rows of the result
    display(airports_df.head())
except Exception as e:
    print(f"An error occurred: {e}")

✈️ Fetching list of large US airports from BigQuery...
✅ Found 639 large airports.


,iata_code,name,latitude_deg,longitude_deg
0,ABQ,Albuquerque International Sunport,35.039976,-106.608925
1,ADW,Joint Base Andrews,38.810799,-76.866997
2,ATL,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101
3,AUS,Austin Bergstrom International Airport,30.197535,-97.662015
4,BDL,Bradley International Airport,41.938510,-72.688066


In [34]:
import requests
import json
import time
from datetime import datetime, timezone

# This helper function cleans the text returned by Gemini to extract the JSON.
def extract_json(text):
    """Extracts a JSON object from a string, cleaning up markdown formatting."""
    try:
        # Gemini often wraps JSON in ```json ... ```
        if '```json' in text:
            text = text.split('```json')[1].split('```')[0]
        return json.loads(text)
    except (json.JSONDecodeError, IndexError) as e:
        print(f"   - Error parsing JSON: {e}")
        return None

# --- 1. Define the Prompt for Gemini ---
# This prompt instructs Gemini to act as an aviation analyst and return
# a structured JSON object, which is much more reliable to parse.
prompt_template = """
As an expert aviation weather analyst, review the following NWS forecast for an airport.
Generate a concise operational alert.

The alert must include:
1.  A one-sentence summary of the weather conditions.
2.  A brief statement on potential operational impacts (e.g., delays from thunderstorms, diversions due to fog, ground stops for high winds).
3.  A severity rating: LOW, MEDIUM, or HIGH.

Format the entire output as a single, clean JSON object with three keys: "summary", "impact", and "severity".

Forecast Data:
{forecast_text}
"""

# --- 2. Main Processing Loop ---
alerts_data = []
print("\n🤖 Starting weather forecast processing for each airport...")

# We'll process the first 5 airports for this example.
# Remove or comment out the ".head(5)" to run for all airports.
for index, airport in airports_df.head(5).iterrows():
    iata = airport['iata_code']
    lat = airport['latitude_deg']
    lon = airport['longitude_deg']

    print(f"\nProcessing {iata} ({lat:.2f}, {lon:.2f})...")

    try:
        # Step A: Get the specific forecast URL from the NWS API
        points_url = f"https://api.weather.gov/points/{lat},{lon}"
        points_response = requests.get(points_url, headers={'Accept': 'application/geo+json'})
        points_response.raise_for_status() # Raises an exception for bad status codes
        forecast_url = points_response.json()['properties']['forecast']

        # Step B: Get the actual forecast data
        forecast_response = requests.get(forecast_url, headers={'Accept': 'application/geo+json'})
        forecast_response.raise_for_status()
        periods = forecast_response.json()['properties']['periods'][:4] # Get the next ~24 hours
        forecast_text = " ".join([p['detailedForecast'] for p in periods])

        # Step C: Send the forecast to Gemini for analysis
        print("   - Sending forecast to Gemini for analysis...")
        full_prompt = prompt_template.format(forecast_text=forecast_text)
        gemini_response = gemini_model.generate_content(full_prompt)

        # Step D: Parse the structured response from Gemini
        alert_json = extract_json(gemini_response.text)

        if alert_json:
            print(f"   - Success! Severity: {alert_json.get('severity')}")
            # Append the structured data to our list for later upload
            alerts_data.append({
                'iata_code': iata,
                'alert_summary': alert_json.get('summary'),
                'operational_impact': alert_json.get('impact'),
                'severity_level': alert_json.get('severity'),
                'raw_forecast': forecast_text,
                'generated_at': datetime.now(timezone.utc)
            })

    except requests.exceptions.RequestException as e:
        print(f"   - Could not retrieve weather data for {iata}: {e}")
    except Exception as e:
        print(f"   - An unexpected error occurred for {iata}: {e}")

    # Be a good API citizen by waiting a second between requests
    time.sleep(1)

print("\n✅ Finished processing all airports.")


🤖 Starting weather forecast processing for each airport...

Processing ABQ (35.04, -106.61)...
   - Sending forecast to Gemini for analysis...
   - Success! Severity: MEDIUM

Processing ADW (38.81, -76.87)...
   - Sending forecast to Gemini for analysis...
   - Success! Severity: MEDIUM

Processing ATL (33.64, -84.43)...
   - Sending forecast to Gemini for analysis...
   - Success! Severity: MEDIUM

Processing AUS (30.20, -97.66)...
   - Sending forecast to Gemini for analysis...
   - Success! Severity: LOW

Processing BDL (41.94, -72.69)...
   - Sending forecast to Gemini for analysis...
   - Success! Severity: MEDIUM

✅ Finished processing all airports.


In [35]:
import pandas as pd

# --- 1. Convert Alert Data to a DataFrame ---
if not alerts_data:
    print("⚠️ No alerts were generated. Skipping BigQuery load.")
else:
    alerts_df = pd.DataFrame(alerts_data)
    print("💾 Preparing to load generated alerts into BigQuery...")
    display(alerts_df)

    # --- 2. Define the Destination Table and Schema ---
    table_id = "qwiklabs-gcp-02-3205ce60aeba.aero_alerts.airport_weather_alerts"
    job_config = bigquery.LoadJobConfig(
        # Specify the schema for the new table. This is best practice.
        schema=[
            bigquery.SchemaField("iata_code", "STRING"),
            bigquery.SchemaField("alert_summary", "STRING"),
            bigquery.SchemaField("operational_impact", "STRING"),
            bigquery.SchemaField("severity_level", "STRING"),
            bigquery.SchemaField("raw_forecast", "STRING"),
            bigquery.SchemaField("generated_at", "TIMESTAMP"),
        ],
        # If the table already exists, overwrite it with the new data.
        write_disposition="WRITE_TRUNCATE",
    )

    # --- 3. Start the BigQuery Load Job ---
    load_job = client.load_table_from_dataframe(
        alerts_df, table_id, job_config=job_config
    )

    load_job.result()  # Wait for the job to complete

    # --- 4. Confirm the Results ---
    destination_table = client.get_table(table_id)
    print(f"🎉 Successfully loaded {destination_table.num_rows} alerts into the table '{table_id}'.")

💾 Preparing to load generated alerts into BigQuery...


,iata_code,alert_summary,operational_impact,severity_level,raw_forecast,generated_at
0,ABQ,"Scattered thunderstorms are expected, particul...",Potential for delays and diversions due to thu...,MEDIUM,Scattered showers and thunderstorms after 3pm....,2025-09-05 17:38:43.780556+00:00
1,ADW,"A chance of afternoon thunderstorms exists, be...",Potential delays from thunderstorms and associ...,MEDIUM,"Sunny, with a high near 88. Southwest wind aro...",2025-09-05 17:38:46.176305+00:00
2,ATL,Scattered showers and thunderstorms are possib...,Possible delays due to thunderstorms impacting...,MEDIUM,"Mostly sunny, with a high near 92. Southwest w...",2025-09-05 17:38:48.438889+00:00
3,AUS,High temperatures with heat indices near 102 a...,Potential for minor delays due to isolated thu...,LOW,"Mostly sunny, with a high near 101. Heat index...",2025-09-05 17:38:50.639261+00:00
4,BDL,Likely afternoon and evening thunderstorms wit...,"Potential for delays due to thunderstorms, pos...",MEDIUM,"Mostly sunny, with a high near 83. South wind ...",2025-09-05 17:38:52.559854+00:00


🎉 Successfully loaded 5 alerts into the table 'qwiklabs-gcp-02-3205ce60aeba.aero_alerts.airport_weather_alerts'.
